In [59]:
from edgar import *
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup

set_identity("ks@gatech.edu")

[16:55:03] INFO     Identity of the Edgar REST client set to [ks@gatech.edu]                            ]8;id=169212;file:///Users/krish/Desktop/finsights/llm/filings/.venv/lib/python3.9/site-packages/edgar/core.py\core.py]8;;\:]8;id=10236;file:///Users/krish/Desktop/finsights/llm/filings/.venv/lib/python3.9/site-packages/edgar/core.py#158\158]8;;\

In [3]:
ticker = "AAPL"

In [120]:
def get_all_filings(ticker, start_date=None, end_date=None):
    # if start_date is None set it to 1 year ago and end_date to today
    if start_date is None and end_date is None:
        end_date = dt.datetime.now()
        start_date = end_date - dt.timedelta(days=365)
    
    print(f"Getting filings for {ticker} between {start_date} and {end_date}")
    
    company = Company(ticker)
    # get all 10-k, 10-q, 8-k (with press release) filings
    filings_df = company.get_filings(form=['10-K', '10-Q', '8-K']).to_pandas()
    
    # set date column to datetime
    filings_df["filing_date"] = pd.to_datetime(filings_df["filing_date"])
    
    # select filings between start_date and end_date
    filings_df = filings_df[(filings_df["filing_date"] >= start_date) & (filings_df["filing_date"] <= end_date)]
    
    # drop extraneous columns
    filings_df.drop(columns=["act", "fileNumber", "isInlineXBRL", "isXBRL", "primaryDocDescription", "acceptanceDateTime", "size", "primaryDocument", "items"], inplace=True)
    
    # sort by form and date
    filings_df.sort_values(by=["form", "filing_date"], inplace=True)
    
    # initialize list to hold all documents
    all_docs = []
    
    # iterate over all filings
    for _, row in filings_df.iterrows():
       filing = find(row["accession_number"])
       filing_obj = filing.obj()
       relevant_items = []
       if filing.form == "10-K":
           relevant_items = ['1', '1A', '5', '6', '7', '7A', '8', '10', '11']
       elif filing.form == "10-Q":
           relevant_items = ['1', '2', '3', '4', '1A', '5']
       elif filing.form == "8-K":
            relevant_items = filing_obj.items
            # remove the "Item " prefix
            relevant_items = [item[5:] for item in relevant_items]
       all_docs.append({
        "title": f'Form {row["form"]} filed on {row["filing_date"]}',
        "url": filing.document.url,
        "relevances": [filing_obj["ITEM "+item] for item in relevant_items]
        }) 
       
    return all_docs

In [121]:
get_all_filings(ticker)

Getting filings for AAPL between 2023-09-29 17:45:36.006776 and 2024-09-28 17:45:36.006776


[{'title': 'Form 10-K filed on 2023-11-03 00:00:00',
  'url': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm',
  'relevances': ['Item 1.    Business\nCompany Background\nThe Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September.\nProducts\niPhone\niPhone® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 15 Pro, iPhone 15, iPhone 14, iPhone 13 and iPhone SE®.\nMac\nMac® is the Company’s line of personal computers based on its macOS® operating system. The Mac line includes laptops MacBook Air® and MacBook Pro®, as well as desktops iMac®, Mac mini®, Mac Studio® and Mac Pro®.\niPad\niPad® is the Company’s line of multipurpose tablets based on its iPadOS® operating system. The iPad line includes iPad Pro®,